In [53]:
import os

from datetime import datetime

import obspy

import numpy as np
import pandas as pd
import plotly.graph_objects as go

from datetime import timedelta
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from obspy.signal.trigger import classic_sta_lta, trigger_onset

In [54]:
data_directory = './data/mars/training/data/'
cat_directory = './data/mars/training/catalogs/'
categories_file = cat_directory + 'Mars_InSight_training_catalog_final.csv'
categories_df = pd.read_csv(categories_file)
categories_df

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid
0,XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.csv,2022-02-03T08:08:27.000000,507.0,evid0005
1,XB.ELYSE.02.BHV.2022-01-02HR04_evid0006.csv,2022-01-02T04:35:30.000000,2130.0,evid0006


In [65]:
def model(row, lta: int = 50, sta: int = 5):
    test_filename = row.filename
    tr = obspy.read(f'{data_directory}{test_filename.replace(".csv", "")}.mseed')[0]
    tr_data = tr.data

    lta_len = (tr_data.shape[0] / tr.stats.sampling_rate) / lta
    sta_len = lta_len / sta
    
    cft = classic_sta_lta(tr_data, int(sta_len * tr.stats.sampling_rate), int(lta_len * tr.stats.sampling_rate))
    thr_on = np.quantile(cft, 0.999)
    thr_off = np.quantile(cft, 0.01)
    on_off = np.array(trigger_onset(cft, thr_on, thr_off))
    
    return on_off[0][0] - lta

In [66]:
def inference(fold_cat, lta: int = 50, sta: int = 5):
    fnames = []
    detection_times = []
    relative_times = []

    for index in range(len(fold_cat)):
        test_filename = fold_cat.iloc[index].filename
        tr = obspy.read(f'{data_directory}{test_filename.replace(".csv", "")}.mseed')[0]
        starttime = tr.stats.starttime.datetime
        tr_data = tr.data
        tr_times = tr.times()

        trigger = int(model(fold_cat.iloc[index], lta, sta))
        true = int(fold_cat.iloc[index]["time_rel(sec)"] * tr.stats.sampling_rate)

        on_time = starttime + timedelta(seconds=tr_times[trigger])
        fnames.append(test_filename)
        detection_times.append(on_time)
        relative_times.append(trigger / tr.stats.sampling_rate)

    detect_df = pd.DataFrame(data={
        'filename': fnames, 
        'time_abs(%Y-%m-%dT%H:%M:%S.%f)': detection_times, 
        'time_rel(sec)': relative_times
    })

    return detect_df

In [67]:
# def kfold_cross_validation(k=2, lta_range=range(10, 200, 5), sta_range=range(5, 50, 1)):
#     kf = KFold(n_splits=k, shuffle=True, random_state=42)
#     best_rmse = float('inf')
#     best_lta, best_sta = None, None
# 
#     # Split the catalog data into k folds
#     for lta in lta_range:
#         for sta in sta_range:
#             rmse_folds = []
# 
#             # Cross-validation loop
#             for train_idx, test_idx in kf.split(categories_df):
#                 train_cat = categories_df.iloc[train_idx]
#                 test_cat = categories_df.iloc[test_idx]
# 
#                 detect_df = inference(test_cat, lta, sta)
# 
#                 mse = mean_squared_error(detect_df['time_rel(sec)'], test_cat['time_rel(sec)'])
#                 rmse = np.sqrt(mse)
#                 rmse_folds.append(rmse)
# 
#             # Average RMSE across folds
#             avg_rmse = np.mean(rmse_folds)
# 
#             if avg_rmse < best_rmse:
#                 best_rmse = avg_rmse
#                 best_lta = lta
#                 best_sta = sta
# 
#             print(f'LTA: {lta}, STA: {sta}, Avg RMSE: {avg_rmse}')
# 
#     print(f'Best RMSE: {best_rmse}')
#     print(f'Best LTA: {best_lta}')
#     print(f'Best STA: {best_sta}')
# 
# # Perform the k-fold cross-validation
# kfold_cross_validation()
# 
# #LTA: 15, STA: 5, Avg RMSE: 44.19999999999996

In [68]:
def inference(lta: int = 50, sta: int = 5, save_folder = './results', save_images=True):
    os.makedirs(save_folder, exist_ok=True)
    os.makedirs(f'{save_folder}/plots', exist_ok=True)

    fnames = []
    detection_times = []
    relative_times = []

    for index in range(len(categories_df)):
        test_filename = categories_df.iloc[index].filename
        tr = obspy.read(f'{data_directory}{test_filename.replace(".csv", "")}.mseed')[0]
        starttime = tr.stats.starttime.datetime
        tr_data = tr.data
        tr_times = tr.times()

        trigger = int(model(categories_df.iloc[index], lta, sta))
        true = int(categories_df.iloc[index]["time_rel(sec)"] * tr.stats.sampling_rate)

        on_time = starttime + timedelta(seconds = tr_times[trigger])
        on_time_str = datetime.strftime(on_time,'%Y-%m-%dT%H:%M:%S.%f')
        fnames.append(test_filename)
        detection_times.append(on_time_str)
        relative_times.append(trigger / tr.stats.sampling_rate)

        if save_images:
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=tr_times, y=tr_data, mode='lines', name='Seismogram'
            ))
            fig.add_vline(x=tr_times[trigger], line=dict(color='red'), annotation_text="Trig. On", annotation_position="top left")
            fig.add_vline(x=tr_times[true], line=dict(color='blue'), annotation_text="True", annotation_position="top left")

            # Customize the layout
            fig.update_layout(
                title="Seismogram with STA/LTA Triggers",
                xaxis_title="Time (s)",
                yaxis_title="Amplitude",
                xaxis_range=[min(tr_times), max(tr_times)],
                height=400,
                width=900
            )
            fig.write_image(os.path.join(f'{save_folder}/plots/{test_filename}.png'))

    detect_df = pd.DataFrame(data = {
        'filename':fnames,
        'time_abs(%Y-%m-%dT%H:%M:%S.%f)': detection_times,
        'time_rel(sec)': relative_times,
    })
    detect_df['evid'] = categories_df['evid']
    
    detect_df = detect_df.sort_values(
        by='evid',
        key=lambda x: x.str.extract('(\d+)$').iloc[:, 0].astype(int)
    )
    detect_df.to_csv(f'{save_folder}/catalog.csv', index=False)

In [69]:
lta = 15
sta = 5
inference(lta, sta, save_folder=f'./data/mars/lta{lta}_sta{sta}', save_images=True)
df = pd.read_csv(f'./data/mars/lta{lta}_sta{sta}/catalog.csv')
mse = mean_squared_error(df['time_rel(sec)'], categories_df['time_rel(sec)'])
rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')
print(f"{rmse / categories_df['time_rel(sec)'].mean() * 100}%")

RMSE: 44.55965103094947
3.37957156093663%
